# 📃 Real Time Order Maker 📃

This notebook uses simpy's realtime API to run the simulation in .... real time!

The only part of the simulation we use here is the `generate_orders` function. We will use this notebook to initiate order during the live workshop exercise.

In [ ]:
#### Run this cell to install simpy before running the rest of the notebook if you are using Google Colab
!pip install simpy

In [ ]:
import numpy as np
import simpy
import random
import uuid # use this for generating IDs for event logging
from contextlib import ExitStack # this is used for creating "with" context managers with arbitrary numbers of concurrent objects
import pandas as pd

import os
if os.name == 'posix':
    linux = True
else:
    linux = False
    import wave
    import pyaudio # for playing sounds

In [ ]:
df_materials = pd.DataFrame(
    [
        {
            'name': 'supercrane',
            'type': 'finished_good',
            'inputs': ['copy_paper','staple'],
            'recipe': ['S_tl','C_n','G1'],
            'sales_price': 24,
        },
        {
            'name': 'birthday_card',
            'type': 'finished_good',
            'inputs': ['folded_card','staple'],
            'recipe': ['F_h','S_tl','G1'],
            'sales_price': 8.5,
        },
        {
            'name': 'holey_folder',
            'type': 'finished_good',
            'inputs': ['folded_card','staple'],
            'recipe': ['S_tl','P_h','P_h','P_h','S_bl'],
            'sales_price': 9.10,
        },
        {
            'name': 'folded_card',
            'type': 'intermediate_product',
            'inputs': ['construction_paper'],
            'recipe': ['F_h'],
        },
        {
            'name': 'triangle',
            'type': 'finished_good',
            'inputs': ['copy_paper'],
            'recipe': ['F_d','G1'],
            'sales_price': 16.2,
        },
        {
            'name': '2_ply_circle',
            'type': 'finished_good',
            'inputs': ['copy_paper'],
            'recipe': ['C_c','G1'],
            'sales_price': 37,
        },
        {
            'name': 'copy_paper',
            'type': 'raw_material',
            'inputs': [],
            'recipe': [],
            'initial_qty': 100
        },
        {
            'name': 'construction_paper',
            'type': 'raw_material',
            'inputs': [],
            'recipe': [],
            'initial_qty': 25
        },
        {
            'name': 'staple',
            'type': 'raw_material',
            'inputs': [],
            'recipe': [],
            'initial_qty': 200
        },
    ]
)


df_operations = pd.DataFrame(
    [
        {
            'name': 'Staple Top Left Corner',
            'id' : 'S_tl',
            'resources': ['stapler'],
        },
        {
            'name': 'Staple Bottom Left Corner',
            'id' : 'S_bl',
            'resources': ['stapler'],
        },
        {
            'name': 'Glue',
            'id':'G1',
            'resources': ['glue'],
        },
        {
            'name': 'Fold Diagonal',
            'id':'F_d',
            'resources': ['folding_station'],
        },
        {
            'name': 'Cut Notch',
            'id':'C_n',
            'resources': ['scissors'],
        },
        {
            'name': 'Cut Circle',
            'id':'C_c',
            'resources': ['scissors'],
        },
        {
            'name': 'Punch Hole',
            'id':'P_h',
            'resources': ['hole_punch'],
        },
        {
            'name': 'Fold in Half',
            'id':'F_h',
            'resources': ['folding_station'],
        },
    ]
)

df_resources = pd.DataFrame(
    [
        {
            'name': 'stapler',
            'num_units' : 1,
            'num_operators': 2,
            'capacity_per_unit': 1,
            'setup_time': 25,
            'cycle_time_mean': 25,
            'cycle_time_std': 4,
        },
        {
            'name': 'glue',
            'num_units' : 2,
            'setup_time': 18,
            'num_operators': 1,
            'cycle_time_mean': 120,
            'cycle_time_std': 30,
        },
        {
            'name': 'scissors',
            'num_units' : 3,
            'setup_time': 18,
            'num_operators': 1,
            'cycle_time_mean': 48,
            'cycle_time_std': 24,
        },
        {
            'name': 'folding_station',
            'num_units' : 99, # we don't actually have 99 units, but since no physical equipment is required for folding, it is only constrained by available operators
            'setup_time': 18,
            'num_operators': 1,
            'cycle_time_mean': 16,
            'cycle_time_std': 8,
        },
        {
            'name': 'hole_punch',
            'num_units' : 1,
            'setup_time': 10,
            'num_operators': 1,
            'cycle_time_mean': 12,
            'cycle_time_std': 4,
        },
    ]
)

In [ ]:
"""
Machine shop example

Covers:

- Interrupts
- Resources: PreemptiveResource

Scenario:
  A workshop has *n* identical machines. A stream of jobs (enough to
  keep the machines busy) arrives. Each machine breaks down
  periodically. Repairs are carried out by one operator. The operator
  has other, less important tasks to perform, too. Broken machines
  preempt these tasks. The operator continues them when he is done
  with the machine repair. The workshop works continuously.

Modify the below so that:

An Order is created;
An Order steps through a sequence of operations, where in each operation:
    The Order requests a Machine
    The Machine requests an operator (repace operator with operator in the script)
    Once the Operator arrives at the machine, the machine and operator are busy for Setup time + Cycle time
    Then the Operation is complete, and the Order moves on to the next operation

"""



RANDOM_SEED = 42
MTTF = 600.0           # Mean time to failure in seconds
REPAIR_TIME = 15.0     # Time it takes to repair a machine in seconds
NUM_OPERATORS = 5      # 
MINUTES = 15              # Simulation time in minutes
SIM_TIME = MINUTES * 60  # Simulation time in seconds
ORDER_INTERVAL = 12   # Time between orders being started in seconds

LOGGING_INTERVAL = 10 # Time between logging

SIM_FACTOR = 1.0 # REAL TIME simulation factor

CHUNK = 1024 # for audio playback

event_log = pd.DataFrame({'Case_ID': pd.Series(dtype='str'),
                   'Activity': pd.Series(dtype='str'),
                   'Timestamp': pd.Series(dtype='float')})

def log_event(case_ID, activity, timestamp):
    """Log an event to the event log."""
    event_log.loc[len(event_log)] = [case_ID, activity, timestamp]
    

def normal_cycle_time(mean,sigma):
    """Return actual processing time for a concrete part."""
    return np.max([0,np.random.normal(mean, sigma)])


def time_to_failure():
    """Return time until next failure for a machine."""
    return np.random.exponential(MTTF)



# Setup and start the simulation
print('Paper Crafting Conglomerate\n\nORDER MAKER\n\n⌛⌛⌛⌛⏳⏳⏳⏳\n')
random.seed(RANDOM_SEED)  # This helps to reproduce the results

# Create an environment and start the setup process
env = simpy.rt.RealtimeEnvironment(factor=SIM_FACTOR,strict=False)

# create orders for the shop

orders = []

def play_sound(soundfile):
    if linux:
        # use this method for Google colab:
        from IPython.display import Audio
        wn = Audio(soundfile, autoplay=True)
        display(wn)
    else:
        with wave.open(soundfile, 'rb') as wf:
            # Instantiate PyAudio and initialize PortAudio system resources (1)
            p = pyaudio.PyAudio()

            # Open stream (2)
            stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                            channels=wf.getnchannels(),
                            rate=wf.getframerate(),
                            output=True)

            # Play samples from the wave file (3)
            while len(data := wf.readframes(CHUNK)):  # Requires Python 3.8+ for :=
                stream.write(data)

            # Close stream (4)
            stream.close()

            # Release PortAudio system resources (5)
            p.terminate()

def print_order_details(row,indent = ''):
    i = len(orders)
    orders.append(row)
    print(f'{indent}Creating order {i} for {row.name.item()} at {env.now = }')
    for input in row.inputs.item():
        print(f'{indent} - Required input: {input}')
        inputs_required = df_materials.loc[(df_materials.name == input) & (df_materials.type == 'intermediate_product'),:]
        if inputs_required.shape[0]>0:
            print(f'{indent} - Input {input} is an INTERMEDIATE PRODUCT')
            print_order_details(inputs_required,indent + '  ') # recursively place an "Order" for the intermediate product
    print(f'{indent} - Order {i} RECIPE:')
    for operation in row.recipe.item():
        op_name = df_operations.loc[df_operations.id == operation,'name'].item()
        print(f'{indent}   - {operation}:  {op_name}')

 
class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *operator* and continues the production
    after the it is repaired.

    A machine has a *name* and a number of *parts_made* thus far.

    """
    def __init__(self, env, name):
        self.env = env
        self.name = name
        env.process(self.break_machine())

    def break_machine(self):
        """Break the machine every now and then."""
        while True:
            yield self.env.timeout(time_to_failure())
            print(f'\nUh oh! Machine {self.name} broke down at {env.now = }\n')    
            #play sad sound
            play_sound('sad-trombone.wav')

def generate_orders(env):
    while True:
        
        print('\n\n')
        row = df_materials.loc[df_materials.type == 'finished_good',:].sample(1)
        print_order_details(row)
        
        # play an alerting sound
        play_sound('triangle_sound.wav')

        yield env.timeout(ORDER_INTERVAL)

env.process(generate_orders(env))

machines = {}
for row in df_resources.itertuples():
    machines[row.name] = Machine(env,row.name)

env.machines = machines

env.run(until=SIM_TIME+.0001) 
